In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path("..").resolve()))

In [2]:
import torch

from models.models import build_model2, build_model
from datasets import cfg
from datasets.loader import DataModule, DataConfig
from train.trainer_v2 import Trainer, TrainConfig
from train.eval import Evaluator
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = cfg.num_classes 

/home/neoph/dev/Train/.tenv312/lib/python3.12/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/neoph/dev/Train/.tenv312/lib/python3.12/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


In [ ]:
import mlflow.pytorch

m0 = build_model("maskrcnn_r50_fpn", int(cfg.num_classes)).to(device)
m0.load_state_dict(torch.load("../weights/maskrcnn_B_ep40.pth", map_location=device))
m0.eval()

m1 = build_model("maskrcnn_attfpn", int(cfg.num_classes)).to(device)
m1.load_state_dict(torch.load("../weights/maskrcnn_attfpn_A2B.pth", map_location=device))
m1.eval()

m3 = build_model("maskrcnn_attfpn", int(cfg.num_classes)).to(device)
m3.load_state_dict(torch.load("../weights/maskrcnn_alternate 2.pth", map_location=device))
m3.eval()

m4 = build_model("maskrcnn_r50_fpn", int(cfg.num_classes)).to(device)
m4.load_state_dict(torch.load("../weights/_optuna.pth", map_location=device))
m4.eval()



In [4]:
import torch
from torch.utils.data import DataLoader

from datasets.base import collate_bb
from datasets import cfg
from train.metrics import get_metrics, compute_map50

DATA_CONF = DataConfig(batch_size=1, num_workers=2)
DATA = DataModule(DATA_CONF, num_channels=3, with_masks=False)

ds_custom = DATA.ds_custom
ds_a_test = DATA.ds_a_test
ds_b_test = DATA.ds_b_test

ldr_custom = torch.utils.data.DataLoader(ds_custom, batch_size=1, shuffle=False, num_workers=DATA_CONF.num_workers, collate_fn=collate_bb)
ldr_a_test = torch.utils.data.DataLoader(ds_a_test, batch_size=1, shuffle=False, num_workers=DATA_CONF.num_workers, collate_fn=collate_bb)
ldr_b_test = torch.utils.data.DataLoader(ds_b_test, batch_size=1, shuffle=False, num_workers=DATA_CONF.num_workers, collate_fn=collate_bb)

In [6]:
print(len(ds_a_test))

1250


In [5]:

def make_loader(ds, nw=2):
    return DataLoader(ds, batch_size=1, shuffle=False, num_workers=nw, collate_fn=collate_bb)

@torch.no_grad()
def collect(model, loader, need_masks: bool):
    model.eval()
    preds, targs = [], []
    for images, targets in loader:
        images = [im.to(device) for im in images]
        outputs = model(images)

        for out, gt in zip(outputs, targets):
            pred = {
                "boxes": out["boxes"].detach().cpu(),
                "scores": out["scores"].detach().cpu(),
                "labels": out["labels"].detach().cpu(),
            }
            if need_masks:
                pred["masks"] = out["masks"][:, 0].detach().cpu().numpy()  # (N,H,W) numpy

            targ = {"boxes": gt["boxes"].cpu()}
            if "labels" in gt and torch.is_tensor(gt["labels"]):
                targ["labels"] = gt["labels"].cpu()
            if need_masks and "masks" in gt and torch.is_tensor(gt["masks"]):
                targ["masks"] = gt["masks"].cpu().numpy()  # numpy

            preds.append(pred)
            targs.append(targ)

    return preds, targs

def eval_custom_class_agnostic(model):
    preds, targs = collect(model, make_loader(ds_custom), need_masks=True)
    for p in preds:
        p["labels"] = torch.ones((len(p["boxes"]),), dtype=torch.int64)
    for t in targs:
        t["labels"] = torch.ones((len(t["boxes"]),), dtype=torch.int64)
    return get_metrics(preds, targs, num_classes=2)

def eval_a_boxes(model):
    preds, targs = collect(model, make_loader(ds_a_test), need_masks=False)
    return {"mAP50": compute_map50(preds, targs)}

def eval_b_full(model):
    preds, targs = collect(model, make_loader(ds_b_test), need_masks=True)
    return get_metrics(preds, targs, num_classes=cfg.num_classes)

def run(model, name):
    model = model.to(device)
    print(f"\n== {name} ==")
    print("custom (class-agnostic):", eval_custom_class_agnostic(model))
    print("A test (boxes only):    ", eval_a_boxes(model))
    print("B test (full):          ", eval_b_full(model))

run(m0, "m0")
run(m1, "m1")
run(m3, "m2")
run(m4, "m2")


== m0 ==
custom (class-agnostic): {'mAP50': 0.3757989704608917, 'AJI': 0.11060012166415403}
A test (boxes only):     {'mAP50': 0.06020812690258026}
B test (full):           {'mAP50': 0.885177493095398, 'AJI': 0.2648232258287683}

== m1 ==
custom (class-agnostic): {'mAP50': 0.5262898206710815, 'AJI': 0.1116790265997602}
A test (boxes only):     {'mAP50': 0.18084953725337982}
B test (full):           {'mAP50': 0.7555909752845764, 'AJI': 0.20839258637420002}

== m2 ==
custom (class-agnostic): {'mAP50': 0.5505885481834412, 'AJI': 0.09008630332630944}
A test (boxes only):     {'mAP50': 0.9805414080619812}
B test (full):           {'mAP50': 0.936837911605835, 'AJI': 0.37167635683699285}

== m2 ==
custom (class-agnostic): {'mAP50': 0.41571369767189026, 'AJI': 0.10524817045479926}
A test (boxes only):     {'mAP50': 0.10649998486042023}
B test (full):           {'mAP50': 0.9350151419639587, 'AJI': 0.2932402153323205}


In [6]:
\

run(m4, "m3")


== m3 ==
custom (class-agnostic): {'mAP50': 0.41571369767189026, 'PQ_all': 0.03146285823130304, 'mPQ': 0.03146285823130304, 'PQ_per_class': array([0.03146286,        nan]), 'AJI': 0.9999999888888891}
A test (boxes only):     {'mAP50': 0.10649998486042023}
B test (full):           {'mAP50': 0.9350151419639587, 'PQ_all': 0.2885027522146265, 'mPQ': 0.29176939666981966, 'PQ_per_class': array([0.10591872, 0.11129663, 0.12349725, 0.18774629, 0.16565509,
       0.13234615, 0.22198509, 0.1908528 , 0.28052954, 0.22487278,
       0.29265537, 0.27836009, 0.4012936 , 0.41681688, 0.36257702,
       0.40239279, 0.42490641, 0.46898002, 0.37624679, 0.36057416,
       0.54179471, 0.41398836, 0.22881358, 0.2883654 ,        nan]), 'AJI': 0.9999999985074628}
